In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from keras import layers

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [ ]:
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

In [ ]:
train_ds, validation_ds, test_ds = tfds.load(
    "cats_vs_dogs",
    split=["train[:40%]", "train[40%:50%]", "train[50%:60%]"],
    as_supervised=True,
)

In [ ]:
size = (150, 150)
batch_size = 32

train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y))

In [ ]:
train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=10)
validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=10)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=10)

In [ ]:
data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
)

In [ ]:
base_model = keras.applications.Xception(
    weights="imagenet",
    input_shape=(150, 150, 3),
    include_top=False,
)

In [ ]:
base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(150, 150, 3))
x = data_augmentation(inputs)

scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(x)

x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)

outputs = keras.layers.Dense(2, activation='softmax')(x)

In [ ]:
model = keras.Model(inputs, outputs)
model.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=10,
    restore_best_weights=True,
    verbose=1
    )

In [ ]:
model.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
epochs = 500
model.fit(train_ds, epochs=epochs, validation_data=validation_ds, callbacks=[callback])

In [ ]:
score = model.evaluate(train_ds, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
image_batch, label_batch = next(iter(test_ds.take(1)))
image_batch = image_batch.numpy()
label_batch = label_batch.numpy()

In [ ]:
predicted_batch = model.predict(image_batch)
predicted_ids = np.argmax(predicted_batch, axis=-1)

In [ ]:
for i in range(0,len(predicted_ids)):
  print(str(predicted_ids[i]) + " - " + str(label_batch[i]))

cm = confusion_matrix(label_batch, predicted_ids)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm_plot_labels = ['Cats','Dogs']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

In [ ]:
base_model.trainable = True
model.summary()

In [ ]:
model.compile(
    optimizer=Adam(1e-6),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
epochs = 500
model.fit(train_ds, epochs=epochs, validation_data=validation_ds, callbacks=[callback])

In [ ]:
score = model.evaluate(train_ds, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
image_batch, label_batch = next(iter(test_ds.take(1)))
image_batch = image_batch.numpy()
label_batch = label_batch.numpy()

In [ ]:
predicted_batch = model.predict(image_batch)
predicted_ids = np.argmax(predicted_batch, axis=-1)

In [ ]:
for i in range(0,len(predicted_ids)):
  print(str(predicted_ids[i]) + " - " + str(label_batch[i]))

In [ ]:
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')